### Задание 4:
Обучить fasttext-классификатор, сравнить качество классификации с предобученными эмбеддингами и обученными с нуля при классификации.


Данные для обучения моделей:

In [1]:
! wget https: // www.dropbox.com / s / t1gs701zvqaxqnk / rusentiment_random_posts.csv

--2022-10-23 19:34:57--  https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/t1gs701zvqaxqnk/rusentiment_random_posts.csv [following]
--2022-10-23 19:34:57--  https://www.dropbox.com/s/raw/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucce7fe2081a1b8df5eacabd0904.dl.dropboxusercontent.com/cd/0/inline/BvZ2aUliSuB2XXsLFwmAaEeksltKDQbxUgi5mDy0Iv81GD5HT_WPOI6Zh-0heCH7R4Rb7NqFbqZpjp8AtkRrkWRjb77OYtqA0ZmvMPUZYLto-zsbkOGihUOGWPPavAgC1vby92Rg2IcZ4EKN-cPB4afd6YelRuR5IO1L2d4eL4FFCQ/file# [following]
--2022-10-23 19:34:58--  https://ucce7fe2081a1b8df5eacabd0904.dl.dropboxusercontent.com/cd/0/inline/BvZ2aUliSuB2XXsLFwmAaEeksltKDQbxUgi5mDy0Iv81GD5HT_WPOI6Zh-

Данные для тестирования моделей:

In [2]:
! wget https: // www.dropbox.com / s / gr4z1x39y1j6dtx / rusentiment_test.csv

--2022-10-23 19:35:05--  https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/gr4z1x39y1j6dtx/rusentiment_test.csv [following]
--2022-10-23 19:35:05--  https://www.dropbox.com/s/raw/gr4z1x39y1j6dtx/rusentiment_test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucccf175e73dcdcbcaffc0d63b47.dl.dropboxusercontent.com/cd/0/inline/BvZw9F8P86GgS16AeMlR_GNnKM3Flljne74Ev94HuHOqjQMgQf3duDcFBakxA--Dcm_BbtIZEhkyb03nkp4QN9aqpVXQNrqVNV3Xv2gCvr0OJhWNzHPZFJO1cnoe5kjUlmUMCG4MzE4xRI4fFNATRpDnzd7z36HDrHi1X2Jy18J99A/file# [following]
--2022-10-23 19:35:06--  https://ucccf175e73dcdcbcaffc0d63b47.dl.dropboxusercontent.com/cd/0/inline/BvZw9F8P86GgS16AeMlR_GNnKM3Flljne74Ev94HuHOqjQMgQf3duDcFBakxA--Dcm_BbtIZEhkyb03nkp

Импортируем нужные библиотеки.

In [336]:
import fasttext
import fasttext.util
import re
import gensim
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

Создаем новые файлы, чтобы туда записать исходные файлы в нужном формате.

In [109]:
with open('random_posts.txt', 'w+', encoding='utf-8') as f:
    pass

with open('test.txt', 'w+', encoding='utf-8') as f:
    pass

In [130]:
labels_names = ['positive', 'negative', 'neutral', 'speech', 'skip']

with open('rusentiment_random_posts.csv', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('random_posts.txt', 'w', encoding='utf-8') as f:
    for line in lines[1:]:
        check = line.split(',')
        if check[0] in labels_names:
            line = re.sub(r'\n', ' ', line)
            new_line = re.split(r',', line, maxsplit=1)
            f.write('\n')
            f.write('__label__')
            f.write(new_line[0])
            f.write(' ')
            f.write(new_line[1])
        else:
            line = re.sub(r'\n', '', line)
            f.write(' ')
            f.write(line)

with open('rusentiment_test.csv', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('test.txt', 'w', encoding='utf-8') as f:
    for line in lines[1:]:
        check = line.split(',')
        if check[0] in labels_names:
            line = re.sub(r'\n', ' ', line)
            new_line = re.split(r',', line, maxsplit=1)
            f.write('\n')
            f.write('__label__')
            f.write(new_line[0])
            f.write(' ')
            f.write(new_line[1])
        else:
            line = re.sub(r'\n', '', line)
            f.write(' ')
            f.write(line)

Удаляем первые пустые строки из файлов:

In [132]:
with open('test.txt', 'r') as fin:
    data = fin.read().splitlines(True)
with open('test.txt', 'w') as fout:
    fout.writelines(data[1:])

with open('random_posts.txt', 'r') as fin:
    data = fin.read().splitlines(True)
with open('random_posts.txt', 'w') as fout:
    fout.writelines(data[1:])

In [133]:
model = fasttext.train_supervised('random_posts.txt')
model.test('test.txt')

Read 0M words
Number of words:  92487
Number of labels: 5
Progress: 100.0% words/sec/thread: 1008400 lr:  0.000000 avg.loss:  0.873607 ETA:   0h 0m 0s


(2967, 0.6147623862487361, 0.6147623862487361)

Далее используем предобученную модель от FastText:
Скачиваем модель: <code>https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz</code>

Создаем отдельно файлы для лейблов и постов

In [156]:
with open('only_posts.txt', 'w+', encoding='utf-8') as f:
    pass

with open('only_labels.txt', 'w+', encoding='utf-8') as f:
    pass

Приводим файлы в нужный вид для работы векторов и классификатора

In [273]:
with open('random_posts.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('only_labels.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        check = line.split()
        check = check[0].split('__label__')
        f.write(check[1] + '\n')

with open('only_posts.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        line = re.sub(r'[^а-яА-я]+', ' ', line)
        check = line.split(' ')
        f.write(' '.join(check[1:]))
        f.write('\n')

In [184]:
model_ = gensim.models.fasttext.load_facebook_vectors('cc.ru.300.bin')  # Загружаем модель

In [318]:
vectors = []  # Создаем пустой список по размеру кол-ва постов

with open('only_posts.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line in lines:
    # post_vector = np.zeros(shape=(300, 300)) # Создадим пустой вектор для последующей суммы векторов
    if line == '\n':
        post_vector = model_[' ']
        vectors.append(post_vector)
        continue
    line = line.split()
    post_vector = model_[line[0]] * 0
    for word in line:
        post_vector += model_[word]

    post_vector /= len(line)
    vectors.append(post_vector)


In [287]:
with open('only_labels.txt', 'r', encoding='utf-8') as f:
    labels = f.readlines()

In [298]:
labels = np.array(labels)

In [315]:
neigh = KNeighborsClassifier()
neigh.fit(vectors, labels)

KNeighborsClassifier()

In [341]:
with open('test.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('only_labels_test.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        check = line.split()
        check = check[0].split('__label__')
        f.write(check[1] + '\n')

with open('only_posts_test.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        line = re.sub(r'[^а-яА-я]+', ' ', line)
        check = line.split(' ')
        f.write(' '.join(check[1:]))
        f.write('\n')


In [342]:
vectors_test = []  # Создаем пустой список по размеру кол-ва постов

with open('only_posts_test.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line in lines:
    if line == '\n':
        post_vector = model_[' ']
        vectors_test.append(post_vector)
        continue
    line = line.split()
    post_vector = model_[line[0]] * 0
    for word in line:
        post_vector += model_[word]

    post_vector /= len(line)
    vectors_test.append(post_vector)


In [351]:
with open('only_labels_test.txt', 'r', encoding='utf-8') as f:
    test_labels = f.readlines()

In [344]:
res = []
for i in vectors_test:
    res.append(neigh.predict([i]))

In [352]:
f1_score(res, test_labels, average='macro')

0.4798556459988844

Как мы можем видеть, качество классификации с предобученными эмбеддингами (**0.6147623862487361**) выше чем с обученными с нуля (**0.4798556459988844**). Разница составляет **0.13490674024985172**

При анализе тональностей очень важен контекст, который крайне зачастую сложно определить машинным путем.